In [ ]:
import os
from dotenv import load_dotenv
import clickhouse_connect

# Load environment variables
load_dotenv()

# --- ClickHouse Configuration ---
CH_HOST = os.getenv('CH_HOST', 'localhost')
CH_PORT = int(os.getenv('CH_PORT', '18123')) # Often 8123 for HTTP, 9000 for native. Check your setup.
CH_USER = os.getenv('CH_USER', 'default')
CH_PASSWORD = os.getenv('CH_PASSWORD', '') # Default to empty string if not set
CH_DATABASE = os.getenv('CH_DATABASE', 'default') # Optional: specify default database

# --- ClickHouse Connection ---
def get_clickhouse_client():
    """
    Creates and returns a ClickHouse client instance.
    Raises ConnectionError if connection fails immediately.
    """
    # Recommended: Use database parameter during client creation
    client = clickhouse_connect.get_client(
        host='localhost',
        port='18123',
        user='default',
        password='chdefault!9',
        database='cn_futures', # Connect directly to the default DB if needed
    )
    return client

In [4]:
client = get_clickhouse_client()
# Perform a quick test query to ensure connectivity
client.query_arrow("SELECT 1")
print(f"✅ ClickHouse client created and connection tested successfully to {CH_HOST}:{CH_PORT}")

✅ ClickHouse client created and connection tested successfully to localhost:18123


pyarrow.Table
count(): uint64 not null
----
count(): [[803010]]

In [13]:
code = 'RB'
query = f"""SELECT
    datetime,
    date,
    instrument,
    -- 使用 -Merge 组合子合并状态，并计算加权平均
    sumMerge(sum_open_oi_state) / sumMerge(sum_oi_state) AS open,
    sumMerge(sum_high_oi_state) / sumMerge(sum_oi_state) AS high,
    sumMerge(sum_low_oi_state) / sumMerge(sum_oi_state) AS low,
    sumMerge(sum_close_oi_state) / sumMerge(sum_oi_state) AS close,
    -- 使用 -Merge 组合子合并 sum 状态
    sumMerge(sum_volume_state) AS total_volume,
    sumMerge(sum_turnover_state) AS total_turnover,
    sumMerge(sum_oi_state) AS total_open_interest -- OI 的总和也是一个有用的指标
FROM cn_futures.agg_bars_1m
where date>20250101 and instrument = '{code}'
GROUP BY datetime, date, instrument -- 按聚合的键分组，以完成状态的最终合并
ORDER BY datetime ASC, instrument ASC"""

client.query_arrow(query)

pyarrow.Table
datetime: uint64 not null
date: uint32 not null
instrument: string not null
open: double not null
high: double not null
low: double not null
close: double not null
total_volume: uint64 not null
total_turnover: double not null
total_open_interest: double not null
----
datetime: [[20250102090100,20250102090200,20250102090300,20250102090400,20250102090500,...,20250506145600,20250506145700,20250506145800,20250506145900,20250506150000]]
date: [[20250102,20250102,20250102,20250102,20250102,...,20250506,20250506,20250506,20250506,20250506]]
instrument: [["RB","RB","RB","RB","RB",...,"RB","RB","RB","RB","RB"]]
open: [[3303.0616063932043,3293.276703391338,3290.05165238781,3286.099404168303,3284.026138518707,...,3078.8938132519706,3079.7065213713718,3079.738034343199,3078.157167892698,3078.7478685704796]]
high: [[3304.6637112673916,3295.3248209789294,3292.4768622732227,3286.3223111776983,3288.0366024171144,...,3079.816560308682,3079.9055546368472,3079.7847171317894,3079.04732976842